In [ ]:
# 必要なライブラリをインストール
! pip install requests certifi pandas matplotlib

# ライブラリのインポート
import pandas as pd
import requests
import certifi
import matplotlib.pyplot as plt
import tempfile

# セキュリティを維持したままファイルをダウンロード
url = 'https://www.rug.nl/ggdc/docs/pwt90.dta'
response = requests.get(url, verify=certifi.where())
response.raise_for_status()

# 一時ファイルにデータを保存
with tempfile.NamedTemporaryFile(delete=False, suffix='.dta') as tmp:
    tmp.write(response.content)
    tmp_path = tmp.name

# pandasでデータを読み込む
pwt_data = pd.read_stata(tmp_path)


In [13]:
# データをダウンロード
url = 'https://www.rug.nl/ggdc/docs/pwt90.dta'
response = requests.get(url, verify=certifi.where())
response.raise_for_status()

# 一時ファイルに保存
with tempfile.NamedTemporaryFile(delete=False, suffix='.dta') as tmp:
    tmp.write(response.content)
    tmp_path = tmp.name

# データを読み込む
pwt_data = pd.read_stata(tmp_path)

# 重要な国のデータをフィルタリングし、特定の期間のデータを選択
important_countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'United States']
data_filtered = pwt_data[(pwt_data['country'].isin(important_countries)) & (pwt_data['year'].between(1960, 2000))]

# 必要な列のみ選択
data_filtered = data_filtered[['country', 'year', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']]

# 追加変数を計算
data_filtered['alpha'] = 1 - data_filtered['labsh']
data_filtered['y_n'] = data_filtered['rgdpna'] / data_filtered['emp']  # 1人当たりGDP
data_filtered['hours'] = data_filtered['emp'] * data_filtered['avh']  # 総労働時間
data_filtered['tfp_term'] = data_filtered['rtfpna'] ** (1 / (1 - data_filtered['alpha']))  # TFP項
data_filtered['cap_term'] = (data_filtered['rkna'] / data_filtered['rgdpna']) ** (data_filtered['alpha'] / (1 - data_filtered['alpha']))  # 資本項
data_filtered['lab_term'] = data_filtered['hours'] / data_filtered['pop']  # 労働項

# 年度ごとの正規化と再計算
data_filtered = data_filtered.sort_values('year').groupby('country').apply(lambda x: x.assign(
    alpha=1 - x['labsh'],
    y_n_shifted=100 * x['y_n'] / x['y_n'].iloc[0],
    tfp_term_shifted=100 * x['tfp_term'] / x['tfp_term'].iloc[0],
    cap_term_shifted=100 * x['cap_term'] / x['cap_term'].iloc[0],
    lab_term_shifted=100 * x['lab_term'] / x['lab_term'].iloc[0]
)).reset_index(drop=True).dropna()



     country  year  y_n_shifted  tfp_term_shifted  cap_term_shifted  \
0  Australia  1960    99.999992        100.000000        100.000000   
1  Australia  1961   101.071815        100.061691        100.519989   
2  Australia  1962   104.900322        104.384750         99.522270   
3  Australia  1963   108.870148        109.063492         98.378540   
4  Australia  1964   111.746109        111.970459         97.879112   

   lab_term_shifted  
0        100.000000  
1         97.757064  
2         97.998841  
3         98.621439  
4         99.894994  


/var/folders/_1/6pq414md0y1_mgwvl5dlnd2h0000gn/T/ipykernel_51932/1122020981.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_filtered = data_filtered.sort_values('year').groupby('country').apply(lambda x: x.assign(
